<a href="https://colab.research.google.com/github/jcsh4326/notebook/blob/master/PageRank_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This is a project for learning TensorFlow at a deeper level and implementing an algorithm from a low level in order to better understanding.


# Page Rank Algorithm
PageRank(PR) is an algorithm used by Google Search to rank websites in their search engine results.

PageRank was named after Larry Page, one of the founders of Google. 

PageRank is a way of measuring the importance of website pages. According to Google:
```
PageRank works by counting the number and quality of links to a page to determine a rough estimate of how important the website is. The underlying assumption is that more important websites are likely to receive more links from other websites.
```

## Algorithm

### Algorithm Outputs
The PageRank algorithm outputs a **probability distribution** used to represent *the likelihood*  that a person randomly clicking on links will arrive at any particular page.

### Simplified Algorithm
Assume:
```
1. A small universe of 4 web pages: A, B, C and D.
2. PageRank is initialized to the same value for all pages.
```


In [0]:
#@title import
import networkx as nx
import matplotlib.pyplot as plt 

In [0]:
#@title show graph example
nodes = ['a', 'b', 'c', 'd']
edges = [('a', 'b'), ('a', 'c'), ('a', 'd'),('b', 'a'), ('b', 'd'), ('c', 'a'), ('c', 'd'), ('d', 'a'), ('d', 'c')]
G = nx.OrderedMultiDiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges, weight=1)

nx.draw(G, with_labels=True)
plt.show()

## Dead Ends and Spider Traps
Dead Ends and Spider Traps are two problems the person randomly clicking on links may get into trouble.

In [0]:
#@title show a graph with Dead Ends Problem
#@markdown <br> a dead end page means the page has no outbound link so once the person arrived at that page he can never leave out.<br> in this example page c is a dead end page.
nodes = ['a', 'b', 'c', 'd']
edges = [('a', 'b'), ('a', 'c'), ('a', 'd'),('b', 'a'), ('b', 'd'), ('d', 'a'), ('d', 'c')]
G = nx.OrderedMultiDiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

nx.draw(G, with_labels=True)
plt.show()

In [0]:
#@title show a graph with Spider Traps Problem
#@markdown <br> a spider trap page means the page has no outbound link but itself.<br> I didn't install pygraphviz and pydot, so I can't draw the self loops.<br>  the page c has a self loop just image it by yourself.
nodes = ['a', 'b', 'c', 'd']
edges = [('a', 'b'), ('a', 'c'), ('a', 'd'),('b', 'a'), ('b', 'd'), ('d', 'a'), ('d', 'c'), ('c', 'c')]
G = nx.DiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

nx.draw(G, with_labels=True)
print(G.nodes_with_selfloops())
plt.show()

But the person is a smart one who knows how to save himself out of the trouble by inputting a new link which may be the same link currently browsing.


# Implement PageRank Algorithm with Python
 1. $D$: out degree matrix of the page rank graph
    * $D_{ij} = \sum_{j=0}^{n}{A_{ij}}$ 
    * $i=j=n$ 
    * $i$ is the row number and $j$ is the colum number
    * $A_{ij}$ is the adjacent matrix of page rank graph    
 2. $T$: the probability of transferring from one page to the other
   * $T_{ji} = A_{ij}/D_{ii}$
 3. $P$: the probability distribution
   * $P$, $P$ is a $n_p$- dimensional column  vector
   * $n_p$ is the number of pages
   * $P_{iter.}$ is the probability distribution of every iteration
   * $P_0$ is the initialization probability distributionl
   * ${P_0}_j = 1/n_p, j \in [0,  n_p - 1]$
 4. $V$: the probability distribution vector
  * $V = T_{ij}P_0P_1...P_{iter. - 1}$
  * at last $ P_{iter. - 1} = P_{iter.}$
 5. to avoid the dead ends and spider traps problems
  * $P' = αT_{ij}P+(1-α)P_0$
  * $V = T_{ij}P'_0P'_1...P'_{iter. - 1}$
  
for convergence we need a max iteration and a tolerance, but in this simple example, we ignore those conditions.

In [0]:
#@title implement without any library
#@markdown init-param <br> np: $n_p$, the number of pages <br>d: α <br> A: the adjacent matrix
#@markdown edges [(a, b), (a, c), (b, a), (b, d), (c, a), (c, d), (d, a), (d, b)]
np = 4
d = 0.8
A = [[0, 1, 1, 1],\
     [1, 0, 0, 1],\
     [0, 0, 1, 0],\
     [0, 1, 1, 0]]

In [0]:
#@markdown compute the out-degree matrix
D = [[0]*np for i in range(np)]
for idx, a in enumerate(A):
  D[idx][idx] = sum(a)  
print(D)

[[3, 0, 0, 0], [0, 2, 0, 0], [0, 0, 1, 0], [0, 0, 0, 2]]


In [0]:
#@markdown compute hte probability of transferring
T = [[0]*np for i in range(np)]
for i, a in enumerate(A):
  for j, _a in enumerate(a):
    T[j][i] = _a/D[i][i]    
print(T)  

[[0.0, 0.5, 0.0, 0.0], [0.3333333333333333, 0.0, 0.0, 0.5], [0.3333333333333333, 0.0, 1.0, 0.5], [0.3333333333333333, 0.5, 0.0, 0.0]]


In [0]:
#@markdown comupte $P_0$
P0 = [[1/np]*1 for i in range(np)]
print(P0)

[[0.25], [0.25], [0.25], [0.25]]


In [0]:
#@title functions
def matrix_multi(A, B):
  r = [[0]*len(B[0]) for i in range(len(A))]
  for i, a in enumerate(A):
    # a is row_i
    for j in range(len(B[0])):
      #r[i][j] = sum(Arow_i * Bcol_j)      
      m = 0
      for k, _a in enumerate(a):
        r[i][j] = r[i][j] + _a * B[k][j]
  return r

def n_multi_matrix(n, A):
  r = [[0] * len(A[0]) for i in range(len(A))]
  for i, a in enumerate(A):
    for j, _a in enumerate(a):
      r[i][j] = n*_a
  return r

def matrix_plus(A, B):
  r = [[0] * len(A[0]) for i in range(len(A))]
  for i, a in enumerate(A):
    for j, b in enumerate(B):
      for k, _a in enumerate(a):
        r[i][k] = _a + b[k]
  return r

def V(d, T, P, P0):
  return matrix_plus(n_multi_matrix(d, matrix_multi(T, P)), n_multi_matrix(1-d, P0))


In [0]:
#@markdown comupte P'
P = P0
p = V(d, T, P0, P0)
max_iter=100
while(p != P and max_iter > 0):
  P = p
  p = V(d, T, p, P0)
  max_iter -= 1
  print(max_iter)
print(p)
  


# Implement PageRank Algorithm with TensorFlow

In [0]:
#@title import
import tensorflow as tf
import numpy as np

In [0]:
#@title matrix multi with tf
with tf.name_scope('inputs'):
  # define placeholder for inputs to network
  T = tf.placeholder(tf.float32, [None, None], name='Transfer_Probability_Matrix')
  P0 = tf.placeholder(tf.float32, [None ,1], name = 'Init_Probability_Distribution')
  D = tf.placeholder(tf.float32, [None, 1], name = "alpha")
  ONE = tf.placeholder(tf.float32, [None, 1], name = "1")
  PRE = tf.placeholder(tf.float32, [None ,1], name = 'Pre_Probability_Distribution')

with tf.name_scope('Probability_Distribution'):
    with tf.name_scope('alpha_TP'):
      W = tf.multiply(D, tf.matmul(T, PRE), name='Weights')
    with tf.name_scope('1_sub_alpha_multi_P'):
      b = tf.multiply(tf.subtract(ONE,D), P0, name='Biases')
    with tf.name_scope('plus'):
      V = tf.add(W, b)
      tf.summary.histogram('probability_distribution', V)
with tf.name_scope('loss'):
  LOSS = tf.reduce_sum(tf.square(tf.subtract(V, PRE)))
  tf.summary.scalar('loss', LOSS)

In [0]:
#@title simulator input
num_p = 4
_d = 0.85
one = np.ones([num_p, 1], dtype=np.float32)
d = np.full((num_p, 1), _d, dtype=np.float32)
p0 = np.full((num_p, 1), 1/num_p, dtype=np.float32)

In [41]:
A = [[0, 1, 1, 1],\
     [1, 0, 0, 1],\
     [0, 0, 1, 0],\
     [0, 1, 1, 0]]
#@markdown compute the out-degree matrix
od = [[0]*num_p for i in range(num_p)]
for idx, a in enumerate(A):
  od[idx][idx] = sum(a)  
print(od)
#@markdown compute hte probability of transferring
t = [[0]*num_p for i in range(num_p)]
for i, a in enumerate(A):
  for j, _a in enumerate(a):
    t[j][i] = _a/od[i][i]    
print(t)  
t = np.asarray(t)
print(t)

[[3, 0, 0, 0], [0, 2, 0, 0], [0, 0, 1, 0], [0, 0, 0, 2]]
[[0.0, 0.5, 0.0, 0.0], [0.3333333333333333, 0.0, 0.0, 0.5], [0.3333333333333333, 0.0, 1.0, 0.5], [0.3333333333333333, 0.5, 0.0, 0.0]]
[[0.         0.5        0.         0.        ]
 [0.33333333 0.         0.         0.5       ]
 [0.33333333 0.         1.         0.5       ]
 [0.33333333 0.5        0.         0.        ]]


In [0]:
sess = tf.Session()
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('logs/', sess.graph)
init = tf.global_variables_initializer()
sess.run(init)

In [44]:
tol = 0.0000001
pre = p0
v = sess.run(V, feed_dict={T:t, P0:p0, D:d, ONE: one, PRE: pre})
loss = sess.run(LOSS, feed_dict={T:t, P0:p0, D:d, ONE: one, PRE: pre})
print(v)
print(loss)
while(loss > tol):
  pre = v
  v = sess.run(V, feed_dict={T:t, P0:p0, D:d, ONE: one, PRE: pre})
  loss = sess.run(LOSS, feed_dict={T:t, P0:p0, D:d, ONE: one, PRE: pre})
  tb = sess.run(merged, feed_dict={T:t, P0:p0, D:d, ONE: one, PRE: pre})
  writer.add_summary(tb, i)
print(loss)  
print(v)

[[0.14375   ]
 [0.21458334]
 [0.42708334]
 [0.21458334]]
0.04515625
5.929944e-08
[[0.0825805 ]
 [0.10599352]
 [0.70543253]
 [0.10599352]]


In [45]:
#@title install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-11-30 07:44:33--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.238.3.58, 34.232.181.106, 35.172.177.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.238.3.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]   5.11M  3.53MB/s    in 1.5s    

2018-11-30 07:44:35 (3.53 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [46]:
#@title ngrok
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://065ce897.ngrok.io
